In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import NearestNeighbors
from scipy import stats as st
import matplotlib.pyplot as plt
from PIL import Image
import scipy

In [11]:
def h(x):
    # Activation function, let's just use Relu
    return np.maximum(0,x)

def convolve(kernels, x_pixels, y_pixels, image_data, print_names = []):
    x_pixels_convolved = x_pixels - kernels[0].shape[0] + 1 # new x pixels
    y_pixels_convolved = y_pixels - kernels[0].shape[1] + 1 # new y pixels
    channels_convolved = kernels.shape[0]                   # how many kernels and, hence, output channels

    # of original image:
    selected_image = image_data

    if len(print_names) > 0:
        image = Image.fromarray((selected_image).astype(np.uint8)).save("original.png")

    convolved_image = np.empty((channels_convolved,x_pixels_convolved,y_pixels_convolved))
    # now let's cycle through each channel and apply appropriate kernels
    for c in range(channels_convolved):
        convolved_image_raw = scipy.signal.convolve2d(selected_image,kernels[c],mode='valid')
            
        for i in range(x_pixels_convolved):
            for j in range(y_pixels_convolved):
                convolved_image[c][i][j] = h(convolved_image_raw[i][j])  

        if len(print_names) > 0:
            cm = plt.get_cmap('bwr')
            colored_image = cm(convolved_image[c]/255)
            image = Image.fromarray((colored_image[:, :, :3] * 255).astype(np.uint8)).save(print_names[c])
        
    return convolved_image

def max_pool(input_image, windowsize, print_names = []):
    x_pixels = int(input_image[0].shape[0]/windowsize) # new x pixels
    y_pixels = int(input_image[0].shape[1]/windowsize) # new y pixels
    channels = int(input_image.shape[0])     # channels
    
    max_pooled_image = np.zeros((channels,x_pixels,y_pixels))
    # now let's cycle through each channel and max pool
    for c in range(channels):
        for i in range(x_pixels):
            for j in range(y_pixels):
                sum = np.zeros((windowsize,windowsize))
                for i2 in range(windowsize):
                    for j2 in range(windowsize):
                        sum[i2][j2] = input_image[c][windowsize*i + i2][windowsize*j + j2]
                max_pooled_image[c][i][j] = np.max(sum)

        if len(print_names) > 0:
            cm = plt.get_cmap('bwr')
            colored_image = cm(max_pooled_image[c]/255)
            image = Image.fromarray((colored_image[:, :, :3] * 255).astype(np.uint8)).save(print_names[c])

    return max_pooled_image

In [12]:
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

print("Total data points, x pixels, y pixels, color channels")
print(x_test.shape)
print(x_train.shape)

Total data points, x pixels, y pixels, color channels
(10000, 28, 28)
(60000, 28, 28)


In [15]:
# Let's define some convolutions
x_pixels = 28
y_pixels = 28
windowsize = 5
number_of_kernels = 3
kernels = np.empty((number_of_kernels,3,3))
# first kernel
kernels[0] = np.array([[1,1,1],
                         [0,0,0],
                         [-1, -1, -1]])
# second kernel
kernels[1] = np.array([[1, 0, -1],
                         [1, 0, -1],
                         [1, 0, -1]])
# third kernel
kernels[2] = np.array([[-1, -1, -1],
                         [-1, 8, -1],
                         [-1, -1, -1]])

In [16]:
convolve(kernels,28,28,x_train[100],["up.png","side.png","middle.png"]);

In [18]:
new_size = (max_pool(convolve(kernels, x_pixels, y_pixels, x_train[0]), windowsize).shape)
print(new_size)

x_train_new = np.empty((x_train.shape[0],(new_size[0]*new_size[1]*new_size[2])))
x_test_new = np.empty((x_test.shape[0],(new_size[0]*new_size[1]*new_size[2])))

print("Starting testing data")
for i in range(len(x_test)):
    if i % 100 ==0:
        print(i)
    convolved_image = convolve(kernels, x_pixels, y_pixels, x_test[i])
    x_test_new[i] = np.ravel(max_pool(convolved_image, windowsize))
print("Starting training data")
for i in range(len(x_train)):
    if i % 100 ==0:
        print(i)
    convolved_image = convolve(kernels, x_pixels, y_pixels, x_train[i])
    x_train_new[i] = np.ravel(max_pool(convolved_image, windowsize))

print("After convolutions:")
print(x_train_new.shape)
print(x_test_new.shape)

(3, 5, 5)
Starting testing data
0
100
200
300
400
500
600
700
800
900
1000
1100
1200


KeyboardInterrupt: 

In [43]:
# kNN
print("Start")
k=10
model = NearestNeighbors(n_neighbors=k)
model.fit(x_train_new,y_train)
test_neighbors = model.kneighbors_graph(x_test_new)
print("Picking " + str(k) + " nearest neighbors, a total of " + str(x_train_new.shape[0]) + " training points with " + str(x_train_new.shape[1]) + " dimensions")

# test_nearest_neighbors looks through test_neighbors and gets the classifications of the k nearest ones
test_nearest_neighbors = np.empty((y_test.shape[0],k))
# y_pred gets the mode of the nearest neighbors
y_pred = np.empty((y_test.shape[0],1))

print("Total number of test points: " + str(x_test.shape[0]))
for i in range(x_test_new.shape[0]):
    if i%100 == 0:
        print("i=" + str(i))
    test_nearest_neighbors[i] = y_train[np.nonzero(test_neighbors.toarray()[i])].reshape((k))
    y_pred[i] = int(st.mode(test_nearest_neighbors[i], keepdims=True).mode)

conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)
print("Accuracy: " + str(np.round(100*conf_matrix.trace()/conf_matrix.sum(),2)) + "%")
print("Random: " + str(np.round(1/conf_matrix.shape[0],5)) + "%")

Start
Picking 10 nearest neighbors, a total of 50000 training points with 108 dimensions
Total number of test points: 10000
i=0
i=100
i=200
i=300
i=400
i=500
i=600
i=700
i=800
i=900
i=1000
i=1100
i=1200
i=1300
i=1400
i=1500
i=1600
i=1700
i=1800
i=1900
i=2000
i=2100
i=2200
i=2300
i=2400
i=2500
i=2600
i=2700
i=2800
i=2900
i=3000
i=3100
i=3200
i=3300
i=3400
i=3500
i=3600
i=3700
i=3800
i=3900
i=4000
i=4100
i=4200
i=4300
i=4400
i=4500
i=4600
i=4700
i=4800
i=4900
i=5000
i=5100
i=5200
i=5300
i=5400
i=5500
i=5600
i=5700
i=5800
i=5900
i=6000
i=6100
i=6200
i=6300
i=6400
i=6500
i=6600
i=6700
i=6800
i=6900
i=7000
i=7100
i=7200
i=7300
i=7400
i=7500
i=7600
i=7700
i=7800
i=7900
i=8000
i=8100
i=8200
i=8300
i=8400
i=8500
i=8600
i=8700
i=8800
i=8900
i=9000
i=9100
i=9200
i=9300
i=9400
i=9500
i=9600
i=9700
i=9800
i=9900
[[469  29  81  16 104  11 154  11 111  14]
 [ 31 402   7  22  79   9 340  15  50  45]
 [ 53   5 334  50 240  39 244  12  16   7]
 [ 37  13 102 151 244  58 342  26  15  12]
 [ 10   7  83  2